In [14]:
import os
import json
import random
import string

from huggingface_hub import InferenceClient
from dotenv import load_dotenv
from datasets import Dataset

from piidd.data_generation.utils import random_string

load_dotenv("../../.env")
client = InferenceClient(token=os.environ["HF_TOKEN"])

In [8]:
inspirational_people = list(set(open("./inspirational_people.txt", "r").read().split("\n")))

first_names = list(set(json.load(open("../../data/first_names.json"))))
bios = Dataset.from_json("../../data/professional_bios.json")
relations = [
    "friend",
    "colleague",
    "family member",
    "mentor",
    "coworker",
    "teammate",
    "acquaintance",
    "partner",
    "classmate",
    "roommate",
    "neighbor",
    "associate",
    "advisor",
    "club member",
    "peer",
    "companion",
    "fellow volunteer",
    "protege",
    "sponsor",
    "confidant",
    "collaborator",
    "mentor",
    "ally",
    "consort",
    "disciple",
    "follower",
    "mentee",
    "subordinate",
    "supervisor",
    "trainee",
    "apprentice",
    "comrade",
]

In [9]:
basic_prompt = open("./essay_instructions/basic_blog_post.txt", "r").read()

In [16]:
def make_full_prompt(example):

    temp = "# Instructions\n\n"

    temp += basic_prompt + "\n\n"

    bio = random.choice(bios["bio"])

    temp += "Write from the standpoint of the following person:\n\n" + bio + "\n\n"

    temp += "## Guidelines\n\n"

    temp += " - Write in 1st person\n"
    temp += " - Mention your website in the middle of the essay and use <<URL>> as a placeholder.\n"

    relation = random.choice(relations)
    rel_name = random.choice(first_names)
    temp += f" - Mention somewhere in the essay how your {relation} {rel_name} helped you.\n"

    famous = random.choice(inspirational_people)

    temp += f" - Mention somewhere in the essay how {famous} affected you.\n"

    temp += " - The essay should be 500-1000 words\n\n"

    temp += "# Essay\n\n"

    return {
        "starting_prompt": "basic_blog_post",
        "rel_name": rel_name,
        "famous": famous,
        "bio": bio,
        "full_prompt": temp
    }

In [27]:
x = make_full_prompt(1)
x

{'starting_prompt': 'basic_blog_post',
 'rel_name': 'Deborah',
 'famous': 'Elie Wiesel',
 'bio': 'A Cloud Virtualization Specialist in the Cloud Computing industry is a highly analytical and detail-oriented individual, dedicated to designing, implementing, and managing virtualized cloud environments. They possess exceptional problem-solving skills and a deep understanding of cloud infrastructure, enabling them to seamlessly integrate various virtualization technologies for optimized resource utilization. Friendly and approachable, they effectively collaborate with cross-functional teams and clients, ensuring smooth transitions to cloud-based systems while maintaining robust security and compliance standards.',
 'full_prompt': '# Instructions\n\nWrite a blog about overcoming a challenge. Talk about what the problem was, how you approached it, what tools you used, how effective your approach was, and what you would do differently in the future.\n\nWrite from the standpoint of the followi

In [15]:
from datasets import Dataset

ds = Dataset.from_dict({"id": [random_string(10) for _ in range(10)]})

In [19]:
oai_key = os.environ["OPENAI_KEY"]

from openai import OpenAI

client = OpenAI(
    api_key=oai_key,
)

In [30]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": x["full_prompt"]},
  ]
)

In [37]:
print(response.choices[0].message.content)

Overcoming Challenge Through Resilience and Collaboration

As a Cloud Virtualization Specialist in the fast-paced realm of Cloud Computing, challenges are not uncommon. The dynamic nature of technology constantly presents hurdles that require analytical thinking, precision, and adaptability to overcome. One particular challenge stands out in my mind as a testament to the power of resilience and collaboration in tackling difficult situations.

The problem at hand was a complex migration project involving the transfer of a large-scale virtualized environment to a new cloud infrastructure. The project timeline was tight, and the stakes were high to ensure a seamless transition without compromising data integrity or system performance. This task required a comprehensive understanding of virtualization technologies, meticulous planning, and effective execution strategies to minimize downtime and potential disruptions to the business operations relying on the cloud services.

Approaching thi

In [41]:
ds2 = Dataset.from_parquet("gpt-3.5-turbo-v1-part2.pq")

Generating train split: 0 examples [00:00, ? examples/s]

In [43]:
ds2[:5]["response"]

["As an aerospace stress engineer, I am no stranger to challenges that test both my technical knowledge and problem-solving skills. One particular obstacle that stands out in my mind is when our team encountered a complex structural issue during the development phase of a new spacecraft component. The problem presented a significant roadblock in our project timeline and required innovative solutions to overcome.\n\nThe challenge stemmed from unexpected vibrations detected during testing, which raised concerns about the component's structural integrity under various operating conditions. As someone deeply invested in the safety and reliability of aerospace vehicles, I knew that addressing this issue was crucial to ensuring the success of our project. It was clear that a comprehensive analysis and redesign were necessary to meet the rigorous standards of the aerospace industry.\n\nApproaching this challenge required a combination of technical expertise, collaboration with cross-functiona

In [42]:
len([x for x in ds2["response"] if "ERROR" in x])

0